In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
!ls "/content/drive/My Drive/Project"
!pip install -U scikit-learn

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
aftertrain.csv	 logregmodel.pkl  test.csv   train.csv
corona_fake.csv  submit.csv	  tfidf.pkl
     |████████████████████████████████| 6.9MB 111kB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def preprocess(df):
  ps= PorterStemmer()
  stopWords = set(stopwords.words('english'))
  train['pro']=' '
  for i in range(len(df['full'])):
    words = word_tokenize(df['full'][i])
    stem=[]
    for w in words:
      if w not in stopWords:
        stem.append(ps.stem(w))
    df.loc[i,'pro']=' '.join(stem)

In [ ]:
train = pd.read_csv('/content/drive/My Drive/Project/train.csv')
test = pd.read_csv('/content/drive/My Drive/Project/test.csv')
corona=pd.read_csv('/content/drive/My Drive/Project/corona_fake.csv')

In [ ]:
print(train['label'])
test['label']  = '2'
test['label']

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


0       2
1       2
2       2
3       2
4       2
       ..
5195    2
5196    2
5197    2
5198    2
5199    2
Name: label, Length: 5200, dtype: object

In [ ]:
test=test.fillna(' ')
train=train.fillna(' ')
corona=corona.fillna(' ')
test['full']=test['title']+' '+test['author']+test['text']
train['full']=train['title']+' '+train['author']+train['text']
corona['full']=corona['text']

In [ ]:
for i in range(len(corona['label'])):
  if corona['label'][i] == "Fake":
    corona['label'][i] = np.int64(1)
  else:
    corona['label'][i] = np.int64(0)

corona_label=np.array(corona['label'],dtype='int')
train_label=np.array(train['label'])

In [ ]:
preprocess(test)
preprocess(corona)
preprocess(train)

In [ ]:
print(train['pro'][0])
print(corona['pro'][0])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train['pro'].values, train_label, random_state=0)   

In [ ]:
#tfidf_vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1,3),max_df=0.5,smooth_idf=True,norm='l2')
tfidf_vectorizer = TfidfVectorizer(analyzer='word', binary=False, decode_error='strict', encoding='utf-8',input='content', lowercase=True, max_df=0.5, max_features=None,min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,smooth_idf=True, stop_words=None, strip_accents=None,sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',tokenizer=None, use_idf=True, vocabulary=None)
tfidf_train = tfidf_vectorizer.fit(train['pro'].values)
tfidf_train_ans = train['label'].values
tfidf_train = tfidf_vectorizer.transform(train['pro'].values)
corona_train= tfidf_vectorizer.transform(corona['pro'].values)
#tfidf_test = tfidf_vectorizer.fit_transform(test['full'].values)

In [ ]:
X_train=tfidf_vectorizer.transform(X_train)
X_test=tfidf_vectorizer.transform(X_test)

In [ ]:
modelNB = MultinomialNB()   #NaiveBayes
modelNB.fit(X_train, y_train)
#print(modelNB.score(corona_train,corona_label))
print('Accuracy of NaiveBayes classifier on training set: {:.2f}'.format(modelNB.score(X_train, y_train)))
print('Accuracy of NaiveBayes classifier on test set: {:.2f}'.format(modelNB.score(X_test, y_test)))

Accuracy of NaiveBayes classifier on training set: 0.97
Accuracy of NaiveBayes classifier on test set: 0.86


In [ ]:
#logreg = LogisticRegression(max_iter=200,random_state=0)
logreg=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,intercept_scaling=1, l1_ratio=None, max_iter=1000,multi_class='auto', n_jobs=None, penalty='l2',random_state=0, solver='lbfgs', tol=0.0001, verbose=0,warm_start=False)
logreg.fit(X_train, y_train)
print('Accuracy of LogisticRegress classifier on training set: {:.3f}'.format(logreg.score(X_train, y_train)))
print('Accuracy of LogisticRegress classifier on test set: {:.3f}'.format(logreg.score(X_test, y_test)))

Accuracy of LogisticRegress classifier on training set: 0.991
Accuracy of LogisticRegress classifier on test set: 0.954


In [ ]:
print('Accuracy of LogisticRegress classifier on COVID dataset : {:.3f}'.format(logreg.score(corona_train, corona_label)))

Accuracy of LogisticRegress classifier on COVID dataset: 0.419


In [ ]:
submit = pd.read_csv('/content/drive/My Drive/Project/submit.csv')
tfdif2 = tfidf_vectorizer.transform(test['pro'].values)
predictions = logreg.predict(tfdif2)
pred = pd.DataFrame(predictions,columns=['label'])
pred['id']=test['id']
pred.groupby('label').count()

,id
label,
0,2733
1,2467


In [ ]:
accuracy=(sum(pred['label'] == submit['label']))/(len(pred['label']))
print(accuracy)

0.6388461538461538


In [ ]:
article=input("Enter article")
ps=PorterStemmer()
stopWords = set(stopwords.words('english'))
words = word_tokenize(article)
stem=[]
for w in words:
  if w not in stopWords:
    stem.append(ps.stem(w))
finalart=' '.join(stem)
final=[]
final.append(finalart)
print(final)
test1=tfidf_vectorizer.transform(final)
if logreg.predict(test1)==0:
  print("News is reliable")
else:
    print("News is unreliable")
#print(logreg.predict(test1))




Enter articleIndia-China Border Clash Updates: The Indian Army has said no personnel are “unaccounted for” following the Galwan clash on Monday night. Reports have suggested that 10 soldiers, including two Majors, were released on Thursday evening, three days after the violent scuffle with Chinese troops on the Ladakh border. However, there is no official word on the capture of Indian soldiers. India lost 20 soldiers in the clashes while a total of 76 personnel have received injuries to various degrees. Ten Indian soldiers were also taken captive by the Chinese Army who were released on Thursday, reported PTI. Tensions are simmering among the Army units stationed in Ladakh following the killings of 20 soldiers. Prime Minister Narendra Modi will be holding an all-party meeting this evening to brief the political parties on the Ladakh clash and the ongoing situation between India and China. The Modi government has come under heavy criticism for the deaths of 20 Indian soldiers at the han

In [ ]:
import pickle
pickle.dump(tfidf_vectorizer,open('/content/drive/My Drive/Project/tfidf.pkl','wb'))

In [ ]:
import joblib 
  
# Save the model as a pickle in a file 
joblib.dump(logreg, '/content/drive/My Drive/Project/logregmodel.pkl')


['/content/drive/My Drive/Project/logregmodel.pkl']

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(smooth_idf=True)),
    ('clf', LogisticRegression()),
])
parameters = {
    'clf__C':[0.001,0.009,0.01,.09,1,5,10,25]
}

grid_search_tune = GridSearchCV(pipeline, parameters)
grid_search_tune.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_tune.best_estimator_.steps)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Best parameters set:
[('tfidf', TfidfVectorizer()), ('clf', LogisticRegression(C=25))]
